## BIG G EXPRESS EDA, STEP 1 - 4/7/2022
### ROLL ON EIGHTEEN WHEELERS

In [3]:
# IMPORT LIBRARIES

import numpy as np
import pandas as pd
import sqlite3    # ONLY IMPORT THIS ONCE FOR A SET OF FILES
from tqdm.notebook import tqdm

### INITIAL DATA IMPORT AND SQLITE QUERYING

In [5]:
# DATABASE CREATION ONLY NEEDS TO BE RUN ONE TIME

db_jfaults = sqlite3.connect('../data/jfaults.sqlite')

for chunk in tqdm(pd.read_csv('../data/J1939Faults.csv',
                              dtype={'EquipmentID': str},
                              chunksize = 10000)): 
    chunk.to_sql('jfaults', 
                 db_jfaults, 
                 if_exists = 'append', 
                 index = False)

0it [00:00, ?it/s]

In [8]:
db_jfaults = sqlite3.connect('../data/jfaults.sqlite')

# 607,336 ROWS FOR active = TRUE vs 1,185,166 TOTAL
query_jfaults = """

    SELECT * 
    FROM jfaults
    WHERE LENGTH(EquipmentID) <= 5 
        AND active = TRUE
        AND 
        (
            (
                latitude NOT BETWEEN (36.0666667 + 0.016667) AND (36.0666667 - 0.016667)
                AND
                longitude NOT BETWEEN (-86.4347222 + 0.016667) AND (-86.4347222 - 0.016667) 
            )
            AND
            (
                latitude NOT BETWEEN (35.5883333 + 0.016667) AND (35.5883333 - 0.016667)
                AND 
                longitude NOT BETWEEN (-86.4438888 + 0.016667) AND (-86.4438888 - 0.016667)
            )
            AND
            (
                latitude NOT BETWEEN (36.1950 + 0.016667) AND (36.1950 - 0.016667)
                AND 
                longitude NOT BETWEEN (-83.174722 + 0.016667) AND (-83.174722 - 0.016667)
            )
        )
    ;
    
"""

sqlite_jfaults = pd.read_sql(query_jfaults, db_jfaults)

In [9]:
sqlite_jfaults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,None,unknown,unknown,unknown,unknown,0,111,17,1,2,None,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,None,None,unknown,unknown,unknown,unknown,11,629,12,1,127,None,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,None,unknown,unknown,unknown,unknown,11,1807,2,1,127,None,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
3,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,None,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,1,1,None,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
4,7,990439,2015-02-21 11:40:52.000,Low (Severity Low) Engine Coolant Level,None,unknown,unknown,unknown,unknown,0,111,17,1,2,None,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29.000


In [10]:
sqlite_jfaults.shape

(607336, 20)

In [11]:
sqlite_jfaults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607336 entries, 0 to 607335
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   RecordID               607336 non-null  int64  
 1   ESS_Id                 607336 non-null  int64  
 2   EventTimeStamp         607336 non-null  object 
 3   eventDescription       573423 non-null  object 
 4   actionDescription      0 non-null       object 
 5   ecuSoftwareVersion     459538 non-null  object 
 6   ecuSerialNumber        435992 non-null  object 
 7   ecuModel               575120 non-null  object 
 8   ecuMake                575120 non-null  object 
 9   ecuSource              607336 non-null  int64  
 10  spn                    607336 non-null  int64  
 11  fmi                    607336 non-null  int64  
 12  active                 607336 non-null  int64  
 13  activeTransitionCount  607336 non-null  int64  
 14  faultValue             0 non-null   

In [ ]:
# TO DO IF INDICES ARE NEEDED

# db.execute('CREATE INDEX average_day_wait ON jfaults(...))')

In [13]:
# 603 ROWS active = TRUE FOR fault 5246 vs 1189 ROWS TOTAL
query_faults_5246_lat_long = """

    SELECT COUNT(*) 
    FROM jfaults
    WHERE spn = '5246' 
        AND active = TRUE
        AND LENGTH(EquipmentID) <= 5 AND 
        (
            (
                latitude NOT BETWEEN (36.0666667 + 0.016667) AND (36.0666667 - 0.016667)
                AND
                longitude NOT BETWEEN (-86.4347222 + 0.016667) AND (-86.4347222 - 0.016667) 
            )
            AND
            (
                latitude NOT BETWEEN (35.5883333 + 0.016667) AND (35.5883333 - 0.016667)
                AND 
                longitude NOT BETWEEN (-86.4438888 + 0.016667) AND (-86.4438888 - 0.016667)
            )
            AND
            (
                latitude NOT BETWEEN (36.1950 + 0.016667) AND (36.1950 - 0.016667)
                AND 
                longitude NOT BETWEEN (-83.174722 + 0.016667) AND (-83.174722 - 0.016667)
            )
        )
    ;
    
"""

sqlite_faults_5246_cnt = pd.read_sql(query_faults_5246_lat_long, db_jfaults)

sqlite_faults_5246_cnt

,COUNT(*)
0,603


In [14]:
jfaults_prepped = sqlite_jfaults.copy()

### DATA CLEAN-UP

In [15]:
jfaults_prepped['EquipmentID'] = jfaults_prepped['EquipmentID']\
                                        .map(lambda x: x.lstrip('R')\
                                         .rstrip('aAbBcC'))

In [16]:
jfaults_prepped['EquipmentID'] = pd.to_numeric(jfaults_prepped['EquipmentID'])

In [17]:
# 2169 ROWS WITH EquipmentID OF LEN(6 +) 
#      - BUT WE TOOK CARE OF REMOVING THESE WITH SQLITE

jfaults_prepped[jfaults_prepped['EquipmentID'] >= 100000]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp


In [9]:
# jfaults_prepped = jfaults_prepped[jfaults_prepped['EquipmentID'] < 100000]

In [18]:
jfaults_prepped.drop('actionDescription', 
                     axis=1, 
                     inplace=True)

In [19]:
jfaults_prepped.drop('faultValue', 
                     axis=1, 
                     inplace=True)

In [20]:
jfaults_prepped.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,1,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,None,unknown,unknown,unknown,unknown,11,629,12,1,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,1,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
3,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,1,1,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
4,7,990439,2015-02-21 11:40:52.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,1,2,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29.000


In [21]:
jfaults_prepped.to_csv('../data/jfaults_prepped.csv', sep = ',')

In [22]:
db_jfaults.close()

#### PROCEED TO big_g_eda_2022-04-07-step2_RK FOR CORRELATIONS, etc.